In [8]:
import pysftp
import os 
import numpy as np
import pandas as pd

from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from datetime import datetime

pd.options.mode.chained_assignment = None  # default='warn'


/Users/joaojesus/opt/anaconda3/envs/geo_plotting/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [22]:
path_to_folder='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/TEMPERATURE/HOURLY/'

list_ = os.listdir(path=path_to_folder)
list_.sort()
print(len(list_))

number_days = len(list_)//12
for i in range(number_days):
    files_day = (list_[1+i*12:i*12 + 12])


7796


In [23]:
# ==========
YEAR = 2015
# ==========
number_days = len(list_)//12
files_list = []
for i in range(number_days):
    files_day = (list_[1+i*12:i*12 + 12])
    files_list.append(files_day)

result = []
for sublist in files_list:
    filtered_sublist = [filename for filename in sublist if filename.startswith(f'{YEAR}')]
    if filtered_sublist:
        result.append(filtered_sublist)

In [24]:
result

[['2015-01-01_01:00:00.npz',
  '2015-01-01_02:00:00.npz',
  '2015-01-01_03:00:00.npz',
  '2015-01-01_04:00:00.npz',
  '2015-01-01_05:00:00.npz',
  '2015-01-01_06:00:00.npz',
  '2015-01-01_07:00:00.npz',
  '2015-01-01_08:00:00.npz',
  '2015-01-01_09:00:00.npz',
  '2015-01-01_10:00:00.npz',
  '2015-01-01_11:00:00.npz'],
 ['2015-01-02_01:00:00.npz',
  '2015-01-02_02:00:00.npz',
  '2015-01-02_03:00:00.npz',
  '2015-01-02_04:00:00.npz',
  '2015-01-02_05:00:00.npz',
  '2015-01-02_06:00:00.npz',
  '2015-01-02_07:00:00.npz',
  '2015-01-02_08:00:00.npz',
  '2015-01-02_09:00:00.npz',
  '2015-01-02_10:00:00.npz',
  '2015-01-02_11:00:00.npz'],
 ['2015-01-03_01:00:00.npz',
  '2015-01-03_02:00:00.npz',
  '2015-01-03_03:00:00.npz',
  '2015-01-03_04:00:00.npz',
  '2015-01-03_05:00:00.npz',
  '2015-01-03_06:00:00.npz',
  '2015-01-03_07:00:00.npz',
  '2015-01-03_08:00:00.npz',
  '2015-01-03_09:00:00.npz',
  '2015-01-03_10:00:00.npz',
  '2015-01-03_11:00:00.npz'],
 ['2015-01-04_01:00:00.npz',
  '2015-01-

In [25]:
# Get Temperature Data]
def generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/WIND_EASTWARD/HOURLY/',
                    geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180},
                    data_name = 'wind_eastward',
                    year_files = []):
    
    grid = map_grid(grid_size=None, source = 'era5')
    wizard = data_wizard()
    res_mean = []
    res_mean_selection = []
    file_names = []
    
    for day in year_files:
        # create a list to hold the dataframes
        array_list = []
        for file in day:
            data = np.load(path + file)['arr_0']
            array_list.append(data)
            data = np.mean(array_list, axis=0)

        select_area_indexer = grid.select_area_indexer(
            min_lat= geo_filtering['min_lat'],
            max_lat= geo_filtering['max_lat'],
            min_lon= geo_filtering['min_lon'],
            max_lon= geo_filtering['max_lat'])

        data_selection = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = data)

        file_names.append(file[:10])
        res_mean.append(np.nanmean(data))
        res_mean_selection.append(np.nanmean(data_selection))

        df = pd.DataFrame(data={
            'dates' : file_names,
            f'{data_name}_mean' : res_mean,
            f'{data_name}_mean_selection' : res_mean_selection
        })
        
        df['dates'] = pd.to_datetime(df['dates'])
        df = df.sort_values(by='dates')
        df = df.reset_index()
        df = df.drop(columns=['index'])

    return df

def find_ssw(df):
    values = df['wind_eastward_mean_selection'].to_list()
    SSW_ = [False]

    for i in range(1,len(values)):
        if values[i-1] >= 0 and values[i] < 0:
            SSW_.append(True)
        else:
            SSW_.append(False)

    df['SSW_'] = SSW_
    df['SSW_flag'] = False

    day_threshold = 5
    for index in df[df['SSW_'] == True].index:
        if len(df.iloc[index:index+day_threshold].loc[df['wind_eastward_mean_selection'] < 0]) >= day_threshold :
            df['SSW_flag'].iloc[index] = True
    return df
    

In [26]:
df_wind = generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/WIND_EASTWARD/HOURLY/',
                    geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180},
                    data_name = 'wind_eastward',
                    year_files =result )

df_temperature = generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/TEMPERATURE/HOURLY/',
                    geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180},
                    data_name = 'temperature',
                    year_files =result )

df = pd.merge(df_wind,df_temperature[['dates','temperature_mean_selection']],how='left', on='dates')

df_ssw = find_ssw(df)
# find the index of the row where col2 is equal to 7
index = df.index[df['SSW_flag'] == True].tolist()

In [27]:
import plotly.graph_objects as go

# df = df_dataset
# Create figure and plot the DataFrame
fig = go.Figure()

# fig.add_trace(go.Scatter(x=df['dates'], y=df['res_filtered'].rolling(3).mean(), name='Res Filtered', mode='markers', marker=dict(color='blue', size=2)))
fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_eastward_mean_selection'], name='wind_eastward_mean_selection', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_mean_selection'], name='Temperature Selection Mean', mode='markers', marker=dict(color='orange', size=3), yaxis='y2'))
fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_mean_selection'].rolling(1).mean(), name='Temperature Selection Mean', line=dict(color='orange'), yaxis='y2'))

# Add a dashed line at y=0
fig.add_shape(type='line', x0=df['dates'].min(), y0=0, x1=df['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

# Find the index of the first negative value
first_negative_index = (df['wind_eastward_mean_selection'] < 0).idxmin()

# Add vertical lines for SSW_flag == True
for i, row in df.iterrows():
    if row.SSW_flag == True:
        fig.add_shape(type='line', x0=row.dates, y0=-50, x1=row.dates, y1=50, line=dict(color='red', dash='dash'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-50, 80],
        title='wind_eastward_mean_selection'
    )
    ,yaxis2=dict(
        range=[190, 250],
        title='Temperature Selection Mean',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude in {YEAR}',
    legend=dict(orientation="h", y=1.2, x=0.5)
)

# Show the plot
fig.show()